In [ ]:
import io
import re
import zipfile
import requests
import frontmatter
from openai import OpenAI
from tqdm.auto import tqdm

from IPython.display import display, Markdown

In [ ]:
with open("./openai_api_key.txt", "r") as f:
    openai_api_key = f.read().strip()

In [ ]:
with open('./example.md', 'r', encoding='utf-8') as f:
    post = frontmatter.load(f)

In [ ]:
post.metadata

In [ ]:
display(Markdown(post.content))

In [ ]:
url = 'https://codeload.github.com/DataTalksClub/faq/zip/refs/heads/main'

In [ ]:
resp = requests.get(url)

In [ ]:
repository_data = []

with zipfile.ZipFile(io.BytesIO(resp.content)) as zf:
    for file_info in zf.infolist():
        filename = file_info.filename.lower()
    
        # Only process markdown files
        if not (filename.endswith('.md') or filename.endswith('.mdx')):
            continue
    
        # Read and parse each file
        with zf.open(file_info) as f_in:
            content = f_in.read()
            post = frontmatter.loads(content)
            data = post.to_dict()
            data['filename'] = filename
            repository_data.append(data)

In [ ]:
len(repository_data)

In [ ]:
def read_repo_data(repo_owner, repo_name):
    """
    Download and parse all markdown files from a GitHub repository.
    
    Args:
        repo_owner: GitHub username or organization
        repo_name: Repository name
    
    Returns:
        List of dictionaries containing file content and metadata
    """
    prefix = 'https://codeload.github.com' 
    url = f'{prefix}/{repo_owner}/{repo_name}/zip/refs/heads/main'
    resp = requests.get(url)

    if resp.status_code != 200:
        raise Exception(f"Failed to download repository: {resp.status_code}")

    repository_data = []
    with zipfile.ZipFile(io.BytesIO(resp.content)) as zf:
        for file_info in zf.infolist():
            filename = file_info.filename
            filename_lower = filename.lower()

            if not (filename_lower.endswith('.md') 
                or filename_lower.endswith('.mdx')):
                continue

            try:
                with zf.open(file_info) as f_in:
                    content = f_in.read().decode('utf-8', errors='ignore')
                    post = frontmatter.loads(content)
                    data = post.to_dict()
                    data['filename'] = filename
                    repository_data.append(data)
            except Exception as e:
                print(f"Error processing {filename}: {e}")
                continue

    return repository_data

In [ ]:
dtc_faq = read_repo_data('DataTalksClub', 'faq')
evidently_docs = read_repo_data('evidentlyai', 'docs')

print(f"FAQ documents: {len(dtc_faq)}")
print(f"Evidently documents: {len(evidently_docs)}")

### Simple chunking

In [ ]:
def sliding_window(seq, size, step):
    if size <= 0 or step <= 0:
        raise ValueError("size and step must be positive")

    n = len(seq)
    result = []
    for i in range(0, n, step):
        chunk = seq[i:i+size]
        result.append({'start': i, 'chunk': chunk})
        if i + size >= n:
            break

    return result

In [ ]:
len(evidently_docs[45]["content"])

In [ ]:
evidently_chunks = []
for doc in evidently_docs:
    doc_copy = doc.copy()
    doc_content = doc_copy.pop("content")
    chunks = sliding_window(doc_content, 2000, 1000)
    for chunk in chunks:
        chunk.update(doc_copy)
    evidently_chunks.extend(chunks)

In [ ]:
len(evidently_chunks)

### Splitting by Sections

In [ ]:
def split_markdown_by_level(text, level=2):
    """
    Split markdown text by a specific header level.
    
    :param text: Markdown text as a string
    :param level: Header level to split on
    :return: List of sections as strings
    """
    # This regex matches markdown headers
    # For level 2, it matches lines starting with "## "
    header_pattern = r'^(#{' + str(level) + r'} )(.+)$'
    pattern = re.compile(header_pattern, re.MULTILINE)

    # Split and keep the headers
    parts = pattern.split(text)

    sections = []
    for i in range(1, len(parts), 3):
        # We step by 3 because regex.split() with
        # capturing groups returns:
        # [before_match, group1, group2, after_match, ...]
        # here group1 is "## ", group2 is the header text
        header = parts[i] + parts[i+1]  # "## " + "Title"
        header = header.strip()

        # Get the content after this header
        content = ""
        if i+2 < len(parts):
            content = parts[i+2].strip()

        if content:
            section = f'{header}\n\n{content}'
        else:
            section = header
        sections.append(section)

    return sections

In [ ]:
evidently_chunks = []
for doc in evidently_docs:
    doc_copy = doc.copy()
    doc_content = doc_copy.pop("content")
    sections = split_markdown_by_level(doc_content, level=2)

    for section in sections:
        section_copy = doc_copy.copy()
        section_copy["section"] = section
        evidently_chunks.extend(section_copy)

In [ ]:
len(evidently_chunks)

### Chunking with LLM

In [ ]:
openai_client = OpenAI(api_key=openai_api_key)

def llm(prompt, model='gpt-4o-mini'):
    messages = [
        {"role": "user", "content": prompt}
    ]

    response = openai_client.responses.create(
        model='gpt-4o-mini',
        input=messages
    )

    return response.output_text

In [ ]:
prompt_template = """
Split the provided document into logical sections
that make sense for a Q&A system.

Each section should be self-contained and cover
a specific topic or concept.

<DOCUMENT>
{document}
</DOCUMENT>

Use this format:

## Section Name

Section content with all relevant details

---

## Another Section Name

Another section content

---
""".strip()

In [ ]:
def intelligent_chunking(text):
    prompt = prompt_template.format(document=text)
    response = llm(prompt)
    sections = response.split("---")
    sections = [s.strip() for s in sections if s.strip()]
    return sections

In [ ]:
evidently_chunks = []

for doc in tqdm(evidently_docs):
    doc_copy = doc.copy()
    doc_content = doc_copy.pop("content")
    if not doc_content:
        continue

    sections = intelligent_chunking(doc_content)

    for section in sections:
        section_copy = doc_copy.copy()
        section_copy["section"] = section
        evidently_chunks.append(section_copy)

In [ ]:
len(evidently_chunks)

In [ ]:
display(Markdown(evidently_chunks[4]["section"]))